Librerias necesarias

In [180]:
import pandas as pd
import os 
import random
import uuid
import string
from faker import Faker
from datetime import datetime, timedelta
from IPython.display import display, Markdown # Libreria unicamente para visualizar datos markdown en jupyter

Es necesario definir donde se almacenaran las bases de datos generadas para mantener un orden

In [181]:
# Carpeta para almacenar las bases de datos generadas
carpeta_database = 'database'

# Crear una carpeta si esta no existe
if not os.path.exists(carpeta_database):
  os.makedirs(carpeta_database)

# Path para guardar el archivo
path_info_clientes = os.path.join(carpeta_database, 'info_clientes.csv')
path_inventario = os.path.join(carpeta_database, 'inventario.csv')
path_demanda = os.path.join(carpeta_database, 'demanda.csv')

# Generacion de base de datos para clientes



In [182]:
def generar_rut():
    # Genera un número aleatorio de 8 dígitos para el cuerpo del RUT 
    cuerpo_rut = str(random.randint(10000000, 20000000))

    # Calcula el dígito verificador utilizando el algoritmo de Módulo 11
    suma = sum(int(cuerpo_rut[i]) * (i % 6 + 2) for i in range(7))
    digito_verificador = 11 - (suma % 11)
    if digito_verificador == 11:
        digito_verificador = 0
    elif digito_verificador == 10:
        digito_verificador = 'K'

    # Combina el cuerpo y el dígito verificador para formar el RUT completo
    rut_completo = f"{cuerpo_rut}-{digito_verificador}"

    return rut_completo

In [183]:
fake = Faker('es_CL')

def generar_cliente():
  rut = generar_rut()
  nombre = fake.name()
  correo = fake.email()
  telefono = fake.phone_number()
  ciudad = fake.city()

  return {"Nombre": nombre, "Rut": rut, "Ciudad": ciudad, "Correo": correo, "Telefono": telefono}

cantidad_clientes = 1000

cliente = [generar_cliente() for _ in range (cantidad_clientes)]

df = pd.DataFrame(cliente)

# Guardar el archivo
df.to_csv(path_info_clientes, index=False)

# Visualizar en markdown
#display(Markdown(df.to_markdown()))

# Generacion de base de datos para inventario

In [184]:
# Listas de tipos de productos y sabores
tipos_productos = ['Torta', 'Queque', 'Brownie', 'Rosca', 'Dona', 'Galleta', 'Magdalena', 'Tarta', 'Cupcake']
sabores = ['Chocolate', 'Frutilla', 'Vainilla', 'Caramelo', 'Nuez', 'Mora', 'Maracuya', 'Limon', 'Canela', 'Coco', 'Frambuesa', 'Pera', 'Plátano', 'Mango', 'Pistacho', 'Café']
tamaños = ['Individual', 'Mini', 'Grande', 'Extra grande']
rellenos = ['Manjar', 'Dulce de leche', 'Crema pastelera', 'Crema de chocolate', 'Crema de frutilla']
cubiertas = ['Mani', 'Almendras', 'Chocolate negro', 'Chocolate blanco', 'Merengue', 'Crema', 'Nuez']
provedores = ['Panadería Fina & Co.', 'Dolce Gusto S.R.L.', 'Repostería Selecta', 'Delicias Artesanas', 'Pastelería Gourmet', 'Pan y Más']

## Genera un ID unico para cada producto segun su tipo

In [185]:

conteo_por_prefijo = {}
def generar_codigo_producto(tipos_producto):
    # Obtén los tres primeros caracteres de la descripción
    prefijo = tipos_producto[:3].upper()

    if prefijo not in conteo_por_prefijo:
        conteo_por_prefijo[prefijo] = 1
    else:
        conteo_por_prefijo[prefijo] += 1

    codigo_str = str(conteo_por_prefijo[prefijo]).zfill(4)

    return f"{prefijo}{codigo_str}"


In [186]:

def generar_descripcion():
  sabor = random.choice(sabores)
  tamaño = random.choice(tamaños)
  cubierta = random.choice(cubiertas)
  relleno = random.choice(rellenos)

  return sabor, tamaño, cubierta, relleno


def generar_producto():
  tipo_producto = random.choice(tipos_productos)
  sabor, tamaño, cubierta, relleno = generar_descripcion()
  descripcion = f"{tipo_producto} {tamaño} sabor {sabor} relleno de {relleno} y cubierto de {cubierta}"
  id_producto = generar_codigo_producto(tipo_producto)
  precio = fake.random_int(min=1000, max=10000)
  provedor = random.choice(provedores)

  return {"Tipo de producto": tipo_producto, "ID Producto": id_producto, "Descripcion": descripcion,"Precio de venta": precio, "Proveedor": provedor}

num_productos = 1000

producto = [generar_producto() for _ in range (num_productos)]

df = pd.DataFrame(producto)

#Guardar el archivo
df.to_csv(path_inventario, index=False)

# Visualizar en markdown
#isplay(Markdown(df.to_markdown()))

# Generacion de base de datos para ventas

## Leer base de datos de clientes
Esto es un paso necesario para porder asociar los clientes ya generados y de los cuales tenemos informacion, con los productos en inventario.

In [187]:
df_clientes = pd.read_csv(path_info_clientes)

In [188]:
df_productos = pd.read_csv(path_inventario)

In [189]:
# Establecer un rango realista para las fechas de compras
fecha_actual = datetime.now()

inicio_rango = fecha_actual - timedelta (days = 180)

Se genera una tabla que contenga una fecha realista para la demanda del producto, un nombre de cliente extraido de la base de datos y un tipo de producto que desee. Cabe destacar que de esta forma y usando la funcion 'random.choice' es que puede existir clientes que demanden mas de un tipo de producto, lo cual es completamente valido y cumple con una situacion realista. 

In [190]:
def generar_demanda():
  fecha_compra = fake.date_between_dates(date_start=inicio_rango, date_end=fecha_actual)
  
  # Selecciona una fila aleatoria del DataFrame de productos
  producto = df_productos.sample(1).iloc[0]
  tipo_producto = producto['Tipo de producto']
  codigo_producto = producto['ID Producto']

  cliente = df_clientes.sample(1).iloc[0]
  nombre_cliente = cliente['Nombre']
  rut = cliente['Rut']
  
  id_compra = uuid.uuid4().hex[:6]

  return {"ID Compra": id_compra, "Fecha de compra": fecha_compra, "Nombre": nombre_cliente,"Rut": rut, "Producto demandado": tipo_producto, "ID Producto": codigo_producto}

num_demanda_clientes = 1000

demanda = [generar_demanda() for _ in range (num_demanda_clientes)]

df = pd.DataFrame(demanda)

# Guardar el archivo
df.to_csv(path_demanda, index=False)

# Visualizar en markdown
#display(Markdown(df.to_markdown()))